In [ ]:
import numpy as np
import qiskit as q
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, QuantumRegister, AncillaRegister
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.cm as cm
import qiskit.circuit.library as qlib
%matplotlib inline
from qiskit import transpile

In [ ]:


Nx = 16       
Ny = 16       
Q = 5

U = 0.3
Re = 400.0
L = Nx-1
dt=1.

cs2 = 1/3   
cvis=U*L/Re
tau = cvis/cs2/dt + 0.5
taup= 1/L/cs2/dt + 0.5
dim = 2

nlatx  = int(np.ceil(np.log2(Nx)))
nlaty  = int(np.ceil(np.log2(Ny)))

nlinks = 3

start_latx = 0
start_laty = start_latx + nlatx
start_links = start_laty + nlaty


nlatx1 = list(range(start_latx, start_laty))
nlaty1 = list(range(start_laty, start_links))
nlinks1 = list(range(start_links, start_links + nlinks))

Qexlink= nlatx+nlaty+1 

rho = np.ones ((Ny, Nx)) 
vor = np.zeros ((Ny, Nx)) 
phi = np.zeros ((Ny, Nx)) 
ux  = np.zeros((Ny, Nx))
uy  = np.zeros((Ny, Nx))
dd  = np.zeros((Ny, Nx))
dvordx= np.zeros((Ny, Nx))
dphidy= np.zeros((Ny, Nx))
Ux  = np.zeros((Ny, Nx))
Uy  = np.zeros((Ny, Nx))

In [ ]:

def top_half(A):
    def convert(val):
        return val+1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)
def bottom_half(A):
    def convert(val):
        return val-1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)

In [ ]:


def lshift(n):                            #1
    circ = QuantumCircuit(n)
    for i in range(n):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1, n)), i)
    return circ

def rshift(n):                              #2
    circ = QuantumCircuit(n)
    for i in reversed(range(n)):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1, n)), i)
    return circ



In [ ]:

def UVbounds():

    ux[ 0, :] =0
    uy[ 0, :] =0    

    ux[-1, :] =U
    uy[-1, :] =0

    ux[:,  0] = 0 
    uy[:,  0] = 0
    
    ux[:, -1] = 0
    uy[:, -1] = 0

    return ux,uy


def pvbounds(phi):

    phi[ 0, :] =0
    phi[-1, :] =0
    vor[ 0, :] =-2.*phi[1,:]
    vor[-1, :] =-2.*phi[-2,:]-2.*U

    phi[:,  0] = 0    
    phi[:, -1] = 0
    vor[:,  0] = -2.*phi[:, 1]   
    vor[:, -1] = -2.*phi[:,-2]

    return phi,vor

def graduv(vor,phi):
    dvordy = (vor[2:, 1:-1] - vor[:-2, 1:-1])/2/dt
    dvordx = (vor[1:-1, 2:] - vor[1:-1, :-2])/2/dt  
#
    dvordx = np.pad(dvordx, ((1, 1), (1, 1)), mode='edge')  
    dvordy = np.pad(dvordy, ((1, 1), (1, 1)), mode='edge')      

    dvordx[:, -1] = (-2.*phi[:,-2] -vor[:, -2])/dt
    dvordx[:,  0] = (vor[:, 1] + 2.*phi[:, 1])/dt

    dvordy[ 0, :] = (vor[ 1, :] + 2.*phi[1,:])/dt    
    dvordy[-1, :] = (-2.*phi[-2,:]-2.*U -vor[-2, :])/dt

    return dvordx,dvordy

def vorf(rho,vor,ux,uy,dvordx,dvordy):   
    f    = np.zeros((Q, Ny, Nx))
    fneq = np.zeros((Q, Ny, Nx)) 
    feq  = np.zeros((Q, Ny, Nx)) 

  
    ex = np.array([0, 1, -1, 0,  0], dtype=np.int_)
    ey = np.array([0, 0,  0, 1, -1], dtype=np.int_)
    wt = np.array([2/6, 1/6, 1/6, 1/6, 1/6])   

    ex_3d = ex.reshape(Q, 1, 1)
    ey_3d = ey.reshape(Q, 1, 1)
    wt_3d = wt.reshape(Q, 1, 1)   

    eu   =  ex_3d * ux[np.newaxis, :, :] + ey_3d * uy[np.newaxis, :, :]
    eed  = ((ex_3d - ux[np.newaxis, :, :]) * dvordx[np.newaxis, :, :] + (ey_3d - uy[np.newaxis, :, :]) * dvordy[np.newaxis, :, :])

    feq [:, :, :] = wt_3d*vor[np.newaxis, :, :] * (1 + eu/cs2)    
    f   [:, :, :] = feq[:, :, :]-(tau-1.)*dt*eed*wt_3d* (1 + eu/cs2)   
    fneq[:, :, :] = f[:, :, :] - feq[:, :, :]

    f[3, 0, :]  = f[4,  0, :]
  
    f[4,-1, :]  = f[3, -1, :]
  
    f[1, :, 0]  = f[2,  :, 0] 
  
    f[2, :,-1]  = f[1,  :,-1]    

    return f.flatten() 

def phif(rho,vor,phi,ux,uy):   
    f    = np.zeros((Q, Ny, Nx))
    fneq = np.zeros((Q, Ny, Nx)) 
    feq  = np.zeros((Q, Ny, Nx)) 

  
    ex = np.array([0, 1, -1, 0,  0], dtype=np.int_)
    ey = np.array([0, 0,  0, 1, -1], dtype=np.int_)
    wt = np.array([2/6, 1/6, 1/6, 1/6, 1/6])   

   
    ex_3d = ex.reshape(Q, 1, 1)
    ey_3d = ey.reshape(Q, 1, 1)
    wt_3d = wt.reshape(Q, 1, 1)   

    eed = ex_3d  *(-uy[np.newaxis, :, :]) + ey_3d* ux[np.newaxis, :, :]+dt*vor[np.newaxis, :, :]    
    feq [:, :, :] = wt_3d*phi[np.newaxis, :, :]   
    f   [:, :, :] = feq[:, :, :]-(taup-1.)*dt*wt_3d*eed+dt*wt_3d*vor[np.newaxis, :, :] 
    fneq[:, :, :] = f[:, :, :] - feq[:, :, :]

    f[3, 0, :]  = f[4,  0, :]
  
    f[4,-1, :]  = f[3, -1, :]
  
    f[1, :, 0]  = f[2,  :, 0] 
  
    f[2, :,-1]  = f[1,  :,-1]   

    return f.flatten() 

In [ ]:
def vorCirc(rho,vor,ux,uy,dvordx,dvordy):
    qTOTAL=Qexlink+nlinks
    q = QuantumRegister(qTOTAL,'q')

    setup = QuantumCircuit(q)
    
    A_diag = vorf(rho,vor,ux,uy,dvordx,dvordy)
       
    zeros = np.zeros(Ny*Nx)
    A_diag = np.concatenate((A_diag, np.zeros(2**(qTOTAL-1) - A_diag.size)))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(qTOTAL-1)
    
    Col1 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col1.diagonal(list(B1_diag), qubit=list(range(qTOTAL - 1)))
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col2.diagonal(list(B2_diag),qubit = list(range(qTOTAL - 1)))
    Col2 = Col2.to_gate(label='c2')
    
    target_qubits = list(range(qTOTAL-1))         
    full_qubits = [qTOTAL-1] + target_qubits      

    setup.append(Col1.control(1, ctrl_state='0'), full_qubits)
    setup.append(Col2.control(1, ctrl_state='1'), full_qubits)

    setup.h(qTOTAL-1)

    setup.barrier()
    L1 = lshift(nlatx).to_gate(label = "1L").control (nlinks,ctrl_state = '001')#1
    R2 = rshift(nlatx).to_gate(label = "2R").control (nlinks,ctrl_state = '010')#2

    L3 = lshift(nlaty).to_gate(label = "3LT").control(nlinks,ctrl_state = '011')#3
    R4 = rshift(nlaty).to_gate(label = "4RT").control(nlinks,ctrl_state = '100')#4

    setup.append(L1,nlinks1 + list(reversed(nlatx1)))
    setup.append(R2,nlinks1 + list(reversed(nlatx1)))
    setup.append(L3,nlinks1 + list(reversed(nlaty1)))
    setup.append(R4,nlinks1 + list(reversed(nlaty1)))  
    setup.barrier()

    for i in range(nlinks): 
        setup.h(qTOTAL-2-i)

    return setup

vorCirc(rho,rho,ux,uy,dd,dd).draw()


In [ ]:

def phiCirc(rho,vor,phi,ux,uy):
    qTOTAL=Qexlink+nlinks
    q = QuantumRegister(qTOTAL,'q')

    setup = QuantumCircuit(q)
    
    A_diag = phif(rho,vor,phi,ux,uy)
       
    zeros = np.zeros(Ny*Nx)
    A_diag = np.concatenate((A_diag, np.zeros(2**(qTOTAL-1) - A_diag.size)))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(qTOTAL-1)
    
    Col1 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col1.diagonal(list(B1_diag), qubit=list(range(qTOTAL - 1)))
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col2.diagonal(list(B2_diag),qubit = list(range(qTOTAL - 1)))
    Col2 = Col2.to_gate(label='c2')
    
    target_qubits = list(range(qTOTAL-1))         
    full_qubits = [qTOTAL-1] + target_qubits      

    setup.append(Col1.control(1, ctrl_state='0'), full_qubits)
    setup.append(Col2.control(1, ctrl_state='1'), full_qubits)

    setup.h(qTOTAL-1)

    setup.barrier()
    L1 = lshift(nlatx).to_gate(label = "1L").control (nlinks,ctrl_state = '001')
    R2 = rshift(nlatx).to_gate(label = "2R").control (nlinks,ctrl_state = '010')

    L3 = lshift(nlaty).to_gate(label = "3LT").control(nlinks,ctrl_state = '011')
    R4 = rshift(nlaty).to_gate(label = "4RT").control(nlinks,ctrl_state = '100')

    setup.append(L1,nlinks1 + list(reversed(nlatx1)))
    setup.append(R2,nlinks1 + list(reversed(nlatx1)))
    setup.append(L3,nlinks1 + list(reversed(nlaty1)))
    setup.append(R4,nlinks1 + list(reversed(nlaty1)))  
    setup.barrier()

    for i in range(nlinks): 
        setup.h(qTOTAL-2-i)

    return setup

phiCirc(rho,rho,rho,ux,uy).draw()

In [ ]:
def vortimestep(rho,vor,ux,uy,dvordx,dvordy):   
    zeros = np.zeros((Ny,Nx))
    vors = np.concatenate((rho,rho,rho,rho,rho,zeros,zeros,zeros)).flatten()    
    vorSV = Statevector(vors).expand([1,0]).evolve(vorCirc(rho,vor,ux,uy,dvordx,dvordy))
    vorAr = np.reshape(np.asarray(vorSV)[:Ny*Nx],(Ny,Nx))
    vorAr = np.real(vorAr)*2**(nlinks/2)

    return np.reshape(vorAr,(Ny,Nx))


def phitimestep(rho,vor,phi,ux,uy):
    zeros = np.zeros((Ny,Nx))
    phis = np.concatenate((rho,rho,rho,rho,rho,zeros,zeros,zeros)).flatten()###    
    phiSV = Statevector(phis).expand([1,0]).evolve(phiCirc(rho,vor,phi,ux,uy))
    phiAr = np.reshape(np.asarray(phiSV)[:Ny*Nx],(Ny,Nx))
    phiAr = np.real(phiAr)*2**(nlinks/2)

    return np.reshape(phiAr,(Ny,Nx))


In [ ]:
def visualize(n, Ux, Uy, allUx, allUy, vor, phi):
    density=1
    x = np.linspace(0, L, Nx)
    y = np.linspace(0, L, Ny)
    X, Y = np.meshgrid(x, y)
    tick_positions = [0, 2.5, 5, 7.5, 10, 12.5, 15]

    fig, ax = plt.subplots()
 
    ax.quiver(X[::density, ::density], Y[::density, ::density], 
                  allUx[n+1][::density, ::density], allUy[n+1][::density, ::density], 
                  scale=10)
    
    
    ax.set_xlim(0, L)  
    ax.set_ylim(0, L)  


    ax.set_xticks(tick_positions)  
    ax.set_yticks(tick_positions)  
    
  
    ax.set_title(f'Vector Field at t={n + 1}')
    ax.set_aspect('equal', adjustable='box')


    plt.savefig(f'16-400dxy/vector{n + 1}.png', bbox_inches='tight')
    plt.close(fig)  

    fig, ax = plt.subplots()
    CS = ax.contour(np.arange(Nx), np.arange(Ny), phi, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')

    CS = ax.contour(X, Y, phi, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')

    plt.savefig(f'16-400dxy/contour_t{n+1}.png', bbox_inches='tight')
    plt.close(fig)  


In [ ]:
allrho = [rho.copy()]
allvor = [vor.copy()]
allphi = [phi.copy()]
allux = [ux.copy()]  
alluy = [uy.copy()]
allUx = [Ux.copy()]  
allUy = [Uy.copy()]

steps = 10000
for n in range(steps):

    ux,uy = allux[-1], alluy[-1]
    rho = allrho[-1]
    vor, phi = allvor[-1], allphi[-1]
    Ux, Uy = allUx[-1], allUy[-1]
    
    dvordx,dvordy = graduv(vor,phi)

    vor  = vortimestep(rho,vor,ux,uy,dvordx,dvordy)
    phi  = phitimestep(rho,vor,phi,ux,uy)
    phi,vor = pvbounds(phi) 
    ux =  np.gradient(phi, axis=0)  
    uy = -np.gradient(phi, axis=1)  
    ux,uy = UVbounds()    

    Ux=ux/U
    Uy=uy/U
    np.savetxt(f'16-400dxy/u_gradient_t{n+1}.csv', Ux, delimiter=',')
    np.savetxt(f'16-400dxy/v_gradient_t{n+1}.csv', Uy, delimiter=',')
    allrho.append(rho.copy())
    allvor.append(vor.copy())    
    allphi.append(phi.copy())            
    allux.append(ux.copy())  
    alluy.append(uy.copy())  
    allUx.append(Ux.copy())  
    allUy.append(Uy.copy())  

    if (n + 1) % 10 == 0:
        visualize(n, Ux, Uy, allUx, allUy, vor, phi)
